In [ ]:
import cairo
import numpy as np


def create_background(
    colors,
    x,
    y,
    line_width_multiplier,
    max_line_distance,
    n_lines_total,
    filename,
    scaling_factors=(1, 1, 1, 1),
):
    """
    Creates an SVG background for my Squarespace thread art website

        colors: list of RGB tuples (I have them plus black)
        x, y: integers, represent image dimensions
        line_width_multiplier: scales the line thickness
        max_line_distance: float in range [0, 1], determines maximum amount lines can protrude
        n_lines_total: integer
    """

    def convert_point_to_coords(p):
        if p < x:
            c = (p, 0)
            s = 3
        elif p < x + y:
            c = (x, p - x)
            s = 0
        elif p < 2 * x + y:
            c = (p - x - y, y)
            s = 1
        else:
            c = (0, p - 2 * x - y)
            s = 2

        return (np.array(c), s)

    with cairo.SVGSurface("color/" + filename + ".svg", x, y) as surface:
        context = cairo.Context(surface)
        context.set_line_width(line_width_multiplier)

        context.rectangle(0, 0, x, y)
        context.set_source_rgb(1, 1, 1)
        context.fill()

        n_lines_per_color = int(n_lines_total / (len(colors) + 1))

        for color in colors + [(0, 0, 0)]:
            context.set_source_rgb(*[c / 255 for c in color])

            line_count = 0

            while line_count < n_lines_per_color:
                p1, p2 = np.random.randint(2 * (x + y), size=2)
                (c1, s1), (c2, s2) = [convert_point_to_coords(p) for p in [p1, p2]]

                if np.any(np.abs(c1 - c2) < 0.01 * max(x, y)):
                    continue

                line_distance = np.random.random() * max_line_distance * scaling_factors[s1]

                c2 = c1 + line_distance * (c2 - c1)

                context.move_to(c1[0], c1[1])
                context.line_to(c2[0], c2[1])

                context.stroke()

                line_count += 1
